In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, concatenate, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from tensorflow.keras.models import load_model
import itertools 
filterwarnings('ignore')
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2022-05-07 11:21:45.782243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 11:21:45.814485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 11:21:45.814668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
import tensorflow as tf
import numpy as np
import os
import random


def set_seed(seed):
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)


seed = 1337
set_seed(seed)

In [4]:
train_dir = "train"
val_dir = "val"
test_dir = "test"
# labels = ['benign', 'malignant']
# label_np = np.array(labels)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input


datagen_train = ImageDataGenerator(width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=preprocess_input)
datagen_val = ImageDataGenerator(preprocessing_function=preprocess_input)
datagen_test = ImageDataGenerator(preprocessing_function=preprocess_input)

batch_size = 64
input_shape = (128, 128)

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)
generator_val = datagen_val.flow_from_directory(directory=val_dir,
                                                target_size=input_shape,
                                                batch_size=batch_size,
                                                shuffle=False)
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

Found 43429 images belonging to 38 classes.
Found 5417 images belonging to 38 classes.
Found 5459 images belonging to 38 classes.


In [6]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
model = ResNet152V2(include_top=False, input_shape=(128, 128, 3), weights='imagenet')
model_name = 'ResNet152V2'

2022-05-07 11:21:47.324083: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-07 11:21:47.324553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 11:21:47.324732: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 11:21:47.324870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
headModel = model.output
headModel = Flatten()(headModel)
headModel = Dense(38, activation="softmax")(headModel)
model = Model(inputs=model.input, outputs=headModel)
# model.summary()

In [ ]:
def print_layer_trainable():
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


# print_layer_trainable()

In [8]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=2e-5)
loss = 'categorical_crossentropy'
metrics = ['accuracy', 'Precision', 'Recall']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [9]:
%%time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 50

callbacks = [
    EarlyStopping(patience=3, monitor='val_accuracy'),
    ModelCheckpoint(f'models/{model_name}.h5',
                    monitor='val_accuracy', save_best_only=True)
]
history = model.fit(x=generator_train,
                    epochs=epochs,
                    validation_data=generator_val,
                    callbacks=callbacks)

Epoch 1/50


2022-05-07 11:22:04.288885: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8303
2022-05-07 11:22:05.843835: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


679/679 [==============================] - 170s 228ms/step - loss: 1.0315 - accuracy: 0.7189 - precision: 0.8695 - recall: 0.6447 - val_loss: 0.3293 - val_accuracy: 0.8992 - val_precision: 0.9238 - val_recall: 0.8822
Epoch 2/50
679/679 [==============================] - 151s 223ms/step - loss: 0.2521 - accuracy: 0.9183 - precision: 0.9391 - recall: 0.9029 - val_loss: 0.1762 - val_accuracy: 0.9457 - val_precision: 0.9544 - val_recall: 0.9394
Epoch 3/50
679/679 [==============================] - 152s 224ms/step - loss: 0.1444 - accuracy: 0.9511 - precision: 0.9608 - recall: 0.9440 - val_loss: 0.1228 - val_accuracy: 0.9633 - val_precision: 0.9681 - val_recall: 0.9583
Epoch 4/50
679/679 [==============================] - 152s 224ms/step - loss: 0.0987 - accuracy: 0.9673 - precision: 0.9725 - recall: 0.9625 - val_loss: 0.1015 - val_accuracy: 0.9701 - val_precision: 0.9742 - val_recall: 0.9682
Epoch 5/50
679/679 [==============================] - 152s 224ms/step - loss: 0.0724 - accuracy: 0.